In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import glob
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
tf.config.optimizer.set_jit(True)

In [ ]:
# 0~887 is only for 2017/10 ~2018/12/31 data use


AIR_Train = np.load('AIR_DomainB_Training.npy')[:887]
AIR_Test = np.load('AIR_DomainB_Testing.npy')

SAT_Train = np.load('SAT_DomainB_Training.npy')[:887]
SAT_Test = np.load('SAT_DomainB_Testing.npy')

# EPA_Train = np.load('EPA_DomainB_Training.npy')
# EPA_Test = np.load('EPA_DomainB_Testing.npy')

train = np.concatenate([AIR_Train,SAT_Train])
test = np.concatenate([AIR_Test,SAT_Test])

In [ ]:
def glob_all(dir_path):
    file_list = glob.glob(os.path.join(dir_path,'*.h5'))
    inside = os.listdir(dir_path)
    for dir_name in inside:
        if os.path.isdir(os.path.join(dir_path,dir_name)):
            file_list.extend(glob_all(os.path.join(dir_path,dir_name)))
    return file_list

def auto_delete_hdf5_save_history(history,path):
    file_list = glob_all(path)
    for i in range(len(file_list)-1):
        file_list.sort()
        if (len(file_list)-1) == 0:
            print('Only had Best Weight , Cound not delete !')
        else:
            os.remove(file_list[i])
            
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(path+'plt-loss.png')
    plt.clf()

def Encoder(channel_number):
    input_img = Input(shape=(348,204,1))
    e1 = Conv2D(16, (3, 3), activation='tanh', padding='same')(input_img)
    e2 = MaxPooling2D((2, 2), padding='same')(e1)
    e3 = Conv2D(8, (3, 3), activation='tanh', padding='same')(e2)
    e4 = MaxPooling2D((2, 2), padding='same')(e3)
    e5 = Conv2D(channel_number, (3, 3), activation='tanh', padding='same')(e4)
    e6 = MaxPooling2D((2, 2), padding='same')(e5)
    e7 = Activation('tanh')(e6)
    return Model(input_img, e7)

def Decoder(channel_number):
    input_img = Input(shape=(44,26,channel_number))
    d1 = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
    d2 = UpSampling2D((2, 2))(d1)
    d3 = Conv2D(16, (3, 3), activation='relu', padding='same')(d2)
    d4 = UpSampling2D((2, 2))(d3)
    d5 = Conv2D(32, (3, 3), activation='relu')(d4)
    d6 = UpSampling2D((2, 2))(d5)
    d7 = Conv2D(1, (3, 3), activation='relu', padding='same')(d6)
    return Model(input_img, d7)

In [ ]:
channel_number = 6; total_training_time = 30

for training_time in range(total_training_time):
    
    path = 'Weight/1718AIRSAT_channel'+str(channel_number)+'/'+str(training_time)+'/'
    if not os.path.exists(path):os.makedirs(path)
    K.clear_session()
    x = Input(shape=(348, 204, 1))

    Encoder_model = Encoder(channel_number)
    Decoder_model = Decoder(channel_number)

    Autoencoder = Model(x,Decoder_model(Encoder_model(x)))
    Autoencoder.compile(loss='mse',optimizer='Adam')

    filepath=path+'/{epoch:04d}-train_{loss:f}-test_{val_loss:f}.h5'
    earlystopper = EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=0.01)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience=15, min_lr = 2e-7, verbose = 1)                   
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto')
    callbacks_list = [earlystopper, reduce_lr, checkpoint]

    history = Autoencoder.fit(train,train, validation_data=(test,test), batch_size=16, 
                              epochs=2000,verbose=1,callbacks=callbacks_list)
    auto_delete_hdf5_save_history(history,path)
    del Encoder_model
    del Decoder_model
    del Autoencoder

In [10]:
train = np.concatenate([AIR_Train,SAT_Train])

channel_number = 6
x = Input(shape=(348, 204, 1))

Encoder_model = Encoder(channel_number)
Decoder_model = Decoder(channel_number)

Autoencoder = Model(x,Decoder_model(Encoder_model(x)))
Autoencoder.load_weights(glob_all('Weight/1718AIRSAT_channel'+str(channel_number))[0])
pred = Encoder_model.predict(train)
np.save('1718AIRSAT_DomainB_Code_Training',pred)

In [ ]:
train = np.concatenate([AIR_Train,SAT_Train])

channel_number = 6
x = Input(shape=(348, 204, 1))

Encoder_model = Encoder(channel_number)
Decoder_model = Decoder(channel_number)

Autoencoder = Model(x,Decoder_model(Encoder_model(x)))
Autoencoder.load_weights(glob_all('Weight/AIRSAT_channel'+str(channel_number))[0])
pred = Encoder_model.predict(train)
np.save('AIRSAT_DomainB_Code_Training',pred)

In [ ]:
train = np.concatenate([AIR_Train,SAT_Train,EPA_Train])

channel_number = 6
x = Input(shape=(348, 204, 1))

Encoder_model = Encoder(channel_number)
Decoder_model = Decoder(channel_number)

Autoencoder = Model(x,Decoder_model(Encoder_model(x)))
Autoencoder.load_weights(glob_all('Weight/AIRSATEPA_channel'+str(channel_number))[0])
pred = Encoder_model.predict(train)
np.save('AIRSATEPA_DomainB_Code_Training',pred)